In [19]:
%matplotlib inline

import os
import sys
import certifi
import unicodedata
import pandas as pd
import numpy as np
import re
import csv
import math
from datetime import datetime
import requests
import time

import scipy.stats as stats
import matplotlib.pyplot as plt

from collections import defaultdict
from scipy.stats.stats import pearsonr

In [20]:
#LOAD TOKENIZER FILES
def loadSentences(filename):
    array = []
    with open(filename) as f:
        ar = []
        for line in f:
            if (str(line) != "[\n") & (str(line) != "]\n"):
                ar.append(re.sub('\n$', '', line))
                #print (line)
            if (str(line) == "]\n"):
                array.append(ar)
                ar = []
            
    f.close()
    return array

#le 406 domande
Question = loadSentences("tokenizer/tokenizerQuestion.txt")

#le 406 risposte
Answer = loadSentences("tokenizer/tokenizerAnswer.txt")

#le 1000 ecc per calcolare lo score
QTest = loadSentences("tokenizer/tokenizerQTest.txt")

# le 3 domande di test
Test = loadSentences("tokenizer/tokenizerTest.txt")

#question + answer
QA = loadSentences("tokenizer/tokenizerQuestionAnswer.txt")

#mettiamo tutto insieme per avere un unico vocabolario
full=Question + Answer + Test + QTest

# GENSIM W2V Model

In [30]:
#Word to Vect
# import modules & set up logging
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 
# train word2vec 
iterN = 10 #how many times the training code will run through the data set to train 
#the neural network (kind of like the number of training epochs).

mc = 1 #specifies the minimum amount of times that the word has to appear in the corpus 
#before it is included in the vocabulary – this allows us to easily eliminate rare words 
#and reduce our vocabulary size

s = 50 #In other words, each word in our vocabulary, after training, will be represented 
#by a 200 length word vector.

job = 4 #parallel workers we would like to work on the data – this will speed up the training process
window = 5 #contorno di 5 parole (default)

In [31]:
#Question
modelQ = gensim.models.Word2Vec(Question, iter = iterN, min_count=mc, size = s)

#Answer
modelA = gensim.models.Word2Vec(Answer, iter = iterN, min_count=mc, size = s)

#Question+Answer
modelQA = gensim.models.Word2Vec(QA, iter = iterN, min_count=mc, size = s)

#Qtest
modelQTest = gensim.models.Word2Vec(QTest, iter = iterN, min_count=mc, size = s)

#test
modelTest = gensim.models.Word2Vec(Test, iter = iterN, min_count=mc, size = s)

#model full, ossia il modello gensim con Q+A+Qtest+test per creare un unico vocabolario
modelFull = gensim.models.Word2Vec(full, iter = iterN, min_count=mc, size = s)

2018-03-13 09:34:15,417 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2018-03-13 09:34:15,418 : INFO : collecting all words and their counts
2018-03-13 09:34:15,420 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-03-13 09:34:15,423 : INFO : collected 995 word types from a corpus of 4267 raw words and 406 sentences
2018-03-13 09:34:15,424 : INFO : Loading a fresh vocabulary
2018-03-13 09:34:15,430 : INFO : min_count=1 retains 995 unique words (100% of original 995, drops 0)
2018-03-13 09:34:15,431 : INFO : min_count=1 leaves 4267 word corpus (100% of original 4267, drops 0)
2018-03-13 09:34:15,437 : INFO : deleting the raw counts dictionary of 995 items
2018-03-13 09:34:15,438 : INFO : sample=0.001 downsamples 73 most-common words
2018-03-13 09:34:15,439 : INFO : downsampling leaves estimated 3043 word corpus (71.3% of prior 4267)
2018-03-13 09:34:15,444 : INFO : estimated required memory for 995 words and 50 dimension

2018-03-13 09:34:15,828 : INFO : EPOCH - 4 : training on 36170 raw words (27698 effective words) took 0.0s, 1031386 effective words/s
2018-03-13 09:34:15,848 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-03-13 09:34:15,851 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-03-13 09:34:15,853 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-03-13 09:34:15,854 : INFO : EPOCH - 5 : training on 36170 raw words (27802 effective words) took 0.0s, 1181268 effective words/s
2018-03-13 09:34:15,874 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-03-13 09:34:15,877 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-03-13 09:34:15,880 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-03-13 09:34:15,880 : INFO : EPOCH - 6 : training on 36170 raw words (27795 effective words) took 0.0s, 1157551 effective words/s
2018-03-13 09:34:15,900 : INFO : worker thread finis

2018-03-13 09:34:16,601 : INFO : collecting all words and their counts
2018-03-13 09:34:16,602 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-03-13 09:34:16,607 : INFO : collected 1505 word types from a corpus of 8958 raw words and 1132 sentences
2018-03-13 09:34:16,608 : INFO : Loading a fresh vocabulary
2018-03-13 09:34:16,617 : INFO : min_count=1 retains 1505 unique words (100% of original 1505, drops 0)
2018-03-13 09:34:16,619 : INFO : min_count=1 leaves 8958 word corpus (100% of original 8958, drops 0)
2018-03-13 09:34:16,628 : INFO : deleting the raw counts dictionary of 1505 items
2018-03-13 09:34:16,629 : INFO : sample=0.001 downsamples 69 most-common words
2018-03-13 09:34:16,630 : INFO : downsampling leaves estimated 6159 word corpus (68.8% of prior 8958)
2018-03-13 09:34:16,637 : INFO : estimated required memory for 1505 words and 50 dimensions: 1354500 bytes
2018-03-13 09:34:16,638 : INFO : resetting layer weights
2018-03-13 09:34:16,669 : I

2018-03-13 09:34:16,863 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-03-13 09:34:16,864 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-03-13 09:34:16,866 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-03-13 09:34:16,867 : INFO : EPOCH - 5 : training on 16 raw words (1 effective words) took 0.0s, 309 effective words/s
2018-03-13 09:34:16,869 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-03-13 09:34:16,870 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-03-13 09:34:16,871 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-03-13 09:34:16,872 : INFO : EPOCH - 6 : training on 16 raw words (4 effective words) took 0.0s, 1193 effective words/s
2018-03-13 09:34:16,874 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-03-13 09:34:16,875 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-03-13 09:34:16,876 :

In [32]:
#salva modello
modelQ.save('model/vector_50/mymodelQ')

modelA.save('model/vector_50/mymodelA')

modelQA.save('model/vector_50/mymodelQA')

modelQTest.save('model/vector_50/mymodelQTest')

modelTest.save('model/vector_50/mymodelTest')

modelFull.save('model/vector_50/mymodelFull')


2018-03-13 09:34:34,677 : INFO : saving Word2Vec object under model/vector_50/mymodelQ, separately None
2018-03-13 09:34:34,679 : INFO : not storing attribute vectors_norm
2018-03-13 09:34:34,680 : INFO : not storing attribute cum_table
2018-03-13 09:34:34,696 : INFO : saved model/vector_50/mymodelQ
2018-03-13 09:34:34,697 : INFO : saving Word2Vec object under model/vector_50/mymodelA, separately None
2018-03-13 09:34:34,698 : INFO : not storing attribute vectors_norm
2018-03-13 09:34:34,699 : INFO : not storing attribute cum_table
2018-03-13 09:34:34,742 : INFO : saved model/vector_50/mymodelA
2018-03-13 09:34:34,743 : INFO : saving Word2Vec object under model/vector_50/mymodelQA, separately None
2018-03-13 09:34:34,744 : INFO : not storing attribute vectors_norm
2018-03-13 09:34:34,744 : INFO : not storing attribute cum_table
2018-03-13 09:34:34,779 : INFO : saved model/vector_50/mymodelQA
2018-03-13 09:34:34,779 : INFO : saving Word2Vec object under model/vector_50/mymodelQTest, sep